# Bioinformatics - Binging Pocket Prediction

## Peter Schöberl (K12325049)

This Notebook will outline my approach to solving the challenge for Bioninformatics 2024.

# Approach

1. **Data Analysis**

   First I downloaded the given files for our challenge. I installed PyMol for visualising the data to get a feeling of how to look at proteins and their corresponding 3D shapes and binding pockets.

2. **Data Preparation**

   For the first trial I decided to try the prediction with P2Rank since it is a well established method which doesn't require training nor preprocessing for running first evaluations.

3. **Model Training**

   If the accuracy is not "good enough" I will further train P2Rank specifically on our training data to then evaluate how the accuracy (in this case measured by the Jaccard index) improves.

4. **Model Evaluation**

   For model evaluation I will first run a test trial on 100 AA from our given training set where we also have the corresponding labeled binding pockets. With this I hope to validate my approach.

5. **Postprocessing**

   To export the predictions as required (see sample_submission) I had to extract the residuals which were assigned to binding pockets from the corresponding csv files.

# P2Rank Installation

1. **Cloning the repository from GIT**

git clone https://github.com/rdk/p2rank.git && cd p2rank

2. **Building the project**

./make.sh

3. **Running the project on its test files**

./unit-tests.sh

4. **Notes for later to set in training mode**

distro/prank       # Standard mode for production

./prank.sh         # Development/training mode


# Testing P2Rank on a single file

For a first trial I wrote a function to simply make predictions for a single pdb file.
Such that I can be sure the program works as expected.

In [46]:
import os
import subprocess
# SINGLE PDB FILE

# Running P2Rank works without preprocessing and without training
# So thats what we will do here as a first try

def run_p2rank_on_file(p2rank_dir, pdb_file, output_dir):

    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # We construct the command that will be called
    prank_executable = os.path.join(p2rank_dir, 'distro', 'prank')
    command = [prank_executable, 'predict', '-f', pdb_file, '-o', output_dir]

    # Run the P2Rank command
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Print the output
    print("P2Rank Output:\n", result.stdout.decode())
    #print("P2Rank Errors:\n", result.stderr.decode())

# Example usage
# p2rank_directory = "/Users/peter/p2rank"
# pdb_file_path = "/Users/peter/VS_Studio/Bioninformatics/test_0/0_protein.pdb"
# output_directory = "/Users/peter/VS_Studio/Bioninformatics/Outputs"

#run_p2rank_on_file(p2rank_directory, pdb_file_path, output_directory)

# Running P2Rank on a directory

The program seems to work fine so I will implement a function that runs predictions over all pdb files that are listed in a directory. The directory path for the output can be specified as input for the function.

In [47]:
# Now since the first test work we will loop over all pdb files in our test directory
import os
import subprocess

def run_p2rank_on_directory(p2rank_dir, pdb_directory, output_dir):

    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # we collect all pdb files as a list
    pdb_files = [f for f in os.listdir(pdb_directory) if f.endswith('.pdb')]

    # then we loop over every single pdb files
    for pdb_file in pdb_files:
        # get the path to the pdb file
        pdb_path = os.path.join(pdb_directory, pdb_file)
        
        # construct the P2Rank command
        prank_executable = os.path.join(p2rank_dir, 'distro', 'prank')
        command = [prank_executable, 'predict', '-f', pdb_path, '-o', output_dir]

        # Run the P2Rank command
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Print the output
        print(f"P2Rank Output for {pdb_file}:\n", result.stdout.decode())
        #print(f"P2Rank Errors for {pdb_file}:\n", result.stderr.decode())

# Example usage
# p2rank_directory = "/Users/peter/p2rank"
# pdb_files_directory = "/Users/peter/VS_Studio/Bioninformatics/test_0"
# output_directory = "/Users/peter/VS_Studio/Bioninformatics/Outputs"

# run_p2rank_on_directory(p2rank_directory, pdb_files_directory, output_directory)

# Extracting predictions
For each structure file in the dataset P2Rank produces several output files. Therefore we need a script which pulls out the relevant information for our submission and collects it in the corresponding format as required (see sample_submission). The key here is to realize that our prediction corresponds to the amino acids which where attributed to a binding pocket by P2Rank.

In [48]:
# We collect our results from the _residues.csv files
# residue_label - This column contains the labels of the residues.
# pocket        - This column indicates which pocket each residue is part of. If the residue is part of a binding pocket, this column will have a non-zero value.

import os
import pandas as pd

def collect_binding_residues(output_dir, submission_file):
    submission_data = []

    for item in os.listdir(output_dir):
        # we only look at files that end with _residues.csv
        if item.endswith('_residues.csv'):
            # we get the id by simply removing the rest of the file name
            pdb_id = item.replace('_protein.pdb_residues.csv', '')
            # then we collect the file path to extract the predicitons
            residues_file_path = os.path.join(output_dir, item)
            df = pd.read_csv(residues_file_path)
            
            # I had to debug some key error because of a missing blank space -.-
            #print(df)
            #print(f"Columns in {residues_file_path}: {df.columns.tolist()}")

            # for proper evaluation we require that predictions are stripped properly
            df.columns = df.columns.str.strip()

            # basically if the residue is part of a binding pocket we add it to our predictions
            #binding_residues = df[df['pocket'] > 0]['residue_label'].tolist()
            binding_residues = df[df['pocket'] > 0].apply(lambda row: f"{row['chain']}_{row['residue_label']}", axis=1).tolist()
            prediction = ' '.join(binding_residues)
            submission_data.append({'id': pdb_id, 'prediction': prediction})

            # now we convert our submission_data to a csv file and save it with the inputted path
            df_submission = pd.DataFrame(submission_data)
            df_submission['prediction'] = df_submission['prediction'].astype(str) # ensures str which is necessary for evaluation
            # Sort by 'id' column
            # for this we need to convert the ids to integers
            df_submission['id'] = pd.to_numeric(df_submission['id'], errors='coerce')
            df_submission.sort_values(by='id', inplace=True)
            df_submission['id'] = df_submission['id'].astype(str) # reconvert part of debugging
            df_submission.to_csv(submission_file, index=False)

# submission_file = "/Users/peter/VS_Studio/Bioninformatics/traintestcsv.csv"
# output_dir = "/Users/peter/VS_Studio/Bioninformatics/Outputs"
# submission_data = collect_binding_residues(output_dir, submission_file)


# Validating the approach

To validate my approach I implemented a function to calculate the jaccard index between to sets. Then I made prediction using P2Rank on the first 100 AA from our training set and compared them to the actual binding residues as stated in the train.csv file. Then row wise I compute the jaccard index and finally take the average jaccard index over all 100 rows as an indicator of how well P2Rank worked on this validation set.

In [49]:
# Since I did this without further training or preprocessing I wanted to get some feeling for the accuracy
# I applied P2Rank on 100 pdb files to calculate the Jaccard Index such that I could estimate my current accuracy


def calculate_jaccard_index(set1, set2):
    """
    Calculate the Jaccard index between two sets.
    """

    intersection = set1.intersection(set2)
    union = set1.union(set2)

    jaccard_index = len(intersection) / len(union)

    return jaccard_index

################################################################################################################################################


def evaluate_predictions_test(predictions_file, actual_file):
    actual_df = pd.read_csv(actual_file).iloc[:100]  # Extract rows 0-99
    predicted_df = pd.read_csv(predictions_file).iloc[:100]  # Extract rows 0-99

    jaccard_indices = []

    # Iterate over each row to calculate the Jaccard index
    for i in range(len(actual_df)):
        actual_residues = set(actual_df.iloc[i, 1].split())
        #print(actual_residues)
        #predicted_residues = set(predicted_df.iloc[i, 1].split())
        #print(predicted_residues)

        # i need to handle the case where predictions are empty thus being handled as float
        predicted_residues_str = predicted_df.iloc[i, 1]
        if pd.isna(predicted_residues_str):
            predicted_residues_str = ''
        predicted_residues = set(predicted_residues_str.split())
        jaccard_index = calculate_jaccard_index(actual_residues, predicted_residues)
        jaccard_indices.append(jaccard_index)

    # Compute the average Jaccard index
    average_jaccard = sum(jaccard_indices) / len(jaccard_indices) if jaccard_indices else 0
    return average_jaccard

# Example usage
# submission_file_path = "/Users/peter/VS_Studio/Bioninformatics/traintestcsv.csv"
# train_file_path = "/Users/peter/VS_Studio/Bioninformatics/train.csv"

# average_jaccard_index = evaluate_predictions_test(submission_file_path, train_file_path)
# print(f"Average Jaccard Index: {average_jaccard_index}")

######## RESULT ########
# Average Jaccard Index: 0.2572362310380812
# This would already be graded with a B which is fine by me so LETS GO

# Running P2Rank on our test dataset

The results from our validation set were promissing, so I decided to apply P2Rank on the whole test set to upload my first submission. We follow these steps: 

1. **run_p2rank_on_directory**

This function will call the P2Rank command for every pdb file in our test directory

2. **collect_binding_residues**

Then we will create a submission file from the outputs using the function we have earlier written to extract our predictions. The returned csv file corresponds to the format required for the upload on the challenge server

3. **model evaluation**

We will upload our first submission to the server and evaluate our score :-)

In [50]:
# Run P2Rank on test directory
p2rank_directory = "/Users/peter/p2rank"
pdb_files_directory = "/Users/peter/VS_Studio/Bioninformatics/test"
output_directory = "/Users/peter/VS_Studio/Bioninformatics/Outputs"

run_p2rank_on_directory(p2rank_directory, pdb_files_directory, output_directory)


# Create submission file from the outputs
submission_file = "/Users/peter/VS_Studio/Bioninformatics/Submission.csv"
output_dir = "/Users/peter/VS_Studio/Bioninformatics/Outputs"

submission_data = collect_binding_residues(output_dir, submission_file)


P2Rank Output for 2940_protein.pdb:
 P2Rank 2.4.2-beta.1

predicting pockets for proteins from dataset [2940_protein.pdb]
processing [2940_protein.pdb] (1/1)
predicting pockets finished in 0 hours 0 minutes 5.394 seconds
results saved to directory [/Users/peter/VS_Studio/Bioninformatics/Outputs]

Finished successfully in 0 hours 0 minutes 5.869 seconds.

P2Rank Output for 1345_protein.pdb:
 P2Rank 2.4.2-beta.1

predicting pockets for proteins from dataset [1345_protein.pdb]
processing [1345_protein.pdb] (1/1)
predicting pockets finished in 0 hours 0 minutes 3.547 seconds
results saved to directory [/Users/peter/VS_Studio/Bioninformatics/Outputs]

Finished successfully in 0 hours 0 minutes 4.056 seconds.

P2Rank Output for 69_protein.pdb:
 P2Rank 2.4.2-beta.1

predicting pockets for proteins from dataset [69_protein.pdb]
processing [69_protein.pdb] (1/1)
predicting pockets finished in 0 hours 0 minutes 3.414 seconds
results saved to directory [/Users/peter/VS_Studio/Bioninformatics/Outp

In [61]:
# So before I simply retrieved all possible binding pockets.
# this time i will apply a probability threshold to extract the binding pockets 

import os
import pandas as pd

def collect_binding_residues_threshold(output_dir, submission_file, threshold):
    submission_data = []

    for item in os.listdir(output_dir):
        # we only look at files that end with _residues.csv
        if item.endswith('_residues.csv'):
            # we get the id by simply removing the rest of the file name
            pdb_id = item.replace('_protein.pdb_residues.csv', '')
            # then we collect the file path to extract the predicitons
            residues_file_path = os.path.join(output_dir, item)
            df = pd.read_csv(residues_file_path)
            
            # I had to debug some key error because of a missing blank space -.-
            #print(df)
            #print(f"Columns in {residues_file_path}: {df.columns.tolist()}")

            # for proper evaluation we require that predictions are stripped properly
            df.columns = df.columns.str.strip()

            # basically if the residue is part of a binding pocket we add it to our predictions
            #binding_residues = df[df['pocket'] > 0]['residue_label'].tolist()
            filtered_residues = df[(df['pocket'] > 0) & (df['probability'] >= threshold)]
            binding_residues = filtered_residues.apply(lambda row: f" {row['chain']}_{row['residue_label']}", axis=1).tolist()

            prediction = ''.join(binding_residues)
            submission_data.append({'id': pdb_id, 'prediction': prediction})

            # now we convert our submission_data to a csv file and save it with the inputted path
            df_submission = pd.DataFrame(submission_data)
            df_submission['prediction'] = df_submission['prediction'].astype(str) # ensures str which is necessary for evaluation
            # Sort by 'id' column
            # for this we need to convert the ids to integers
            df_submission['id'] = pd.to_numeric(df_submission['id'], errors='coerce')
            df_submission.sort_values(by='id', inplace=True)
            df_submission['id'] = df_submission['id'].astype(str) # reconvert part of debugging
            df_submission.to_csv(submission_file, index=False, sep=',')

# submission_file = "/Users/peter/VS_Studio/Bioninformatics/traintestcsv.csv"
# output_dir = "/Users/peter/VS_Studio/Bioninformatics/Outputs"
# submission_data = collect_binding_residues(output_dir, submission_file)


In [63]:
submission_file = "/Users/peter/VS_Studio/Bioninformatics/Peter_Schoeberl_Submission.csv"
output_dir = "/Users/peter/VS_Studio/Bioninformatics/Outputs"
submission_data = collect_binding_residues_threshold(output_dir, submission_file, 0.2)

In [76]:
# So before I simply retrieved all possible binding pockets or applied a threshold
# this time i simply pull out from pdb_predictions the most probable pocket

import os
import pandas as pd
import csv

def extract_first_row_residue_ids(file_path):
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Strip spaces from keys and get the 'residue_ids'
            clean_row = {key.strip(): value for key, value in row.items()}
            #print(clean_row)
            # directly return predictions from first most probable pocket
            return clean_row.get('residue_ids', None)
    # if there is no prediction predict none
    return ''

def collect_binding_residues_most_probable(output_dir, submission_file):
    submission_data = []

    for item in os.listdir(output_dir):
        # we only look at files that end with _residues.csv
        if item.endswith('_predictions.csv'):
            # we get the id by simply removing the rest of the file name
            pdb_id = item.replace('_protein.pdb_predictions.csv', '')
            # then we collect the file path to extract the predicitons
            residues_file_path = os.path.join(output_dir, item)
            # and simply return the most probable pocket with its corresponding ids
            prediction = extract_first_row_residue_ids(residues_file_path)
            #print(prediction)
            # then we already add it to our data
            submission_data.append({'id': pdb_id, 'prediction': prediction})

            # now we convert our submission_data to a csv file and save it with the inputted path
            df_submission = pd.DataFrame(submission_data)
            df_submission['prediction'] = df_submission['prediction'].astype(str) # ensures str which is necessary for evaluation
            # Sort by 'id' column
            # for this we need to convert the ids to integers
            df_submission['id'] = pd.to_numeric(df_submission['id'], errors='coerce')
            df_submission.sort_values(by='id', inplace=True)
            df_submission['id'] = df_submission['id'].astype(str) # reconvert part of debugging
            df_submission.to_csv(submission_file, index=False, sep=',')

# submission_file = "/Users/peter/VS_Studio/Bioninformatics/traintestcsv.csv"
# output_dir = "/Users/peter/VS_Studio/Bioninformatics/Outputs"
# submission_data = collect_binding_residues(output_dir, submission_file)


In [77]:
submission_file = "/Users/peter/VS_Studio/Bioninformatics/Peter_Schoeberl_Submission.csv"
output_dir = "/Users/peter/VS_Studio/Bioninformatics/Outputs"
submission_data = collect_binding_residues_most_probable(output_dir, submission_file)

# Evaluation

depending on the extraction I approach I got the following results:

- All possible pockets 		

    =           0.195 score

- Pockets with probability over 0.5 

	=           0.155 score

- Pockets with probability over 0.2  

	=           0.195 score

- The one most probable pocket	    

    =           **0.261 score**
